# DoWhy example on ihdp (Infant Health and Development Program) dataset

In [1]:
# importing required libraries
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

C:\Users\kang_\.conda\envs\data_science\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


### Loading Data

In [2]:
data= pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
for i in range(1,26):
    col.append("x"+str(i))
data.columns = col
data = data.astype({"treatment":'bool'}, copy=False)
data.head()

treatment  y_factual  y_cfactual       mu0       mu1        x1        x2  \
0       True   5.599916    4.318780  3.268256  6.854457 -0.528603 -0.343455   
1      False   6.875856    7.856495  6.636059  7.562718 -1.736945 -1.802002   
2      False   2.996273    6.633952  1.570536  6.121617 -0.807451 -0.202946   
3      False   1.366206    5.697239  1.244738  5.889125  0.390083  0.596582   
4      False   1.963538    6.202582  1.685048  6.191994 -1.045229 -0.602710   

         x3        x4        x5  ...  x16  x17  x18  x19  x20  x21  x22  x23  \
0  1.128554  0.161703 -0.316603  ...    1    1    1    1    0    0    0    0   
1  0.383828  2.244320 -0.629189  ...    1    1    1    1    0    0    0    0   
2 -0.360898 -0.879606  0.808706  ...    1    0    1    1    0    0    0    0   
3 -1.850350 -0.879606 -0.004017  ...    1    0    1    1    0    0    0    0   
4  0.011465  0.161703  0.683672  ...    1    1    1    1    0    0    0    0   

   x24  x25  
0    0    0  
1    0    0  
2    0    0  
3    0    0  
4    0    0  

[5 rows x 30 columns]

### 1.Model

In [3]:
# Create a causal model from the data and given common causes.
xs = ""
for i in range(1,26):
    xs += ("x"+str(i)+"+")
    
model=CausalModel(
        data = data,
        treatment='treatment',
        outcome='y_factual',
        common_causes=xs.split('+')
        )


INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treatment'] on outcome ['y_factual']


### 2.Identify

In [4]:
#Identify the causal effect
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:[]


Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor1 (Default)
Estimand expression:
     d                                                                        
────────────(Expectation(y_factual|x18,x4,x23,x13,x25,x10,x12,x16,x21,x19,x22,
d[treatment]                                                                  

                                                 
x14,x2,x6,x24,x9,x7,x17,x20,x8,x15,x3,x11,x1,x5))
                                                 
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x18,x4,x23,x13,x25,x10,x12,x16,x21,x19,x22,x14,x2,x6,x24,x9,x7,x17,x20,x8,x15,x3,x11,x1,x5,U) = P(y_factual|treatment,x18,x4,x23,x13,x25,x10,x12,x16,x21,x19,x22,x14,x2,x6,x24,x9,x7,x17,x20,x8,x15,x3,x11,x1,x5)

### Estimand : 2
Estimand name: iv
No such variable found!

### Estimand : 3
Estimand name: frontdoor
No such variable found!



### 3. Estimate (using different methods)

#### 3.1 Using Linear Regression

In [5]:
# Estimate the causal effect and compare it with Average Treatment Effect
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True
)

print(estimate)

print("Causal Estimate is " + str(estimate.value))
data_1 = data[data["treatment"]==1]
data_0 = data[data["treatment"]==0]

print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))


INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator


*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

## Realized estimand
b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
Target units: ate

## Estimate
Mean value: 3.928671750872716
p-value: [1.58915682e-156]

Causal Estimate is 3.928671750872716
ATE 4.021121012430829


#### 3.2 Using Propensity Score Matching

In [6]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_matching"
)

print("Causal Estimate is " + str(estimate.value))

print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))


INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Causal Estimate is 3.9791388232170393
ATE 4.021121012430829


#### 3.3 Using Propensity Score Stratification

In [7]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification", method_params={'num_strata':50, 'clipping_threshold':5}
)

print("Causal Estimate is " + str(estimate.value))
print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))



INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Causal Estimate is 3.4550471588628207
ATE 4.021121012430829


#### 3.4 Using Propensity Score Weighting

In [8]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_weighting"
)

print("Causal Estimate is " + str(estimate.value))

print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))


INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5


Causal Estimate is 3.409737824402919
ATE 4.021121012430829


C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


### 4. Refute
##### Refute the obtained estimate using multiple robustness checks.
##### 4.1 Adding a random common cause

In [9]:
refute_results=model.refute_estimate(identified_estimand, estimate,
        method_name="random_common_cause")
print(refute_results)

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5+w_random


Refute: Add a Random Common Cause
Estimated effect:3.409737824402919
New effect:3.3996373299212976



C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


##### 4.2 Using a placebo treatment

In [10]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo)

INFO:dowhy.causal_refuters.placebo_treatment_refuter:Refutation over 100 simulated datasets of permute treatment
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y

INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y

INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y

INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y

INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~placebo+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y

Refute: Use a Placebo Treatment
Estimated effect:3.409737824402919
New effect:-0.025302368452393798
p value:0.45999999999999996



#### 4.3 Data Subset Refuter

In [11]:
res_subset=model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter", subset_fraction=0.9)
print(res_subset)

INFO:dowhy.causal_refuters.data_subset_refuter:Refutation over 0.9 simulated datasets of size 672.3000000000001 each
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please

INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:

INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:

INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:

INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:

INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x18+x4+x23+x13+x25+x10+x12+x16+x21+x19+x22+x14+x2+x6+x24+x9+x7+x17+x20+x8+x15+x3+x11+x1+x5
C:\Users\kang_\.conda\envs\data_science\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:

Refute: Use a subset of data
Estimated effect:3.409737824402919
New effect:3.3687011284370567
p value:0.33999999999999997

